## Calibration of model parameters (BLACK and SCHOLES, MERTON)
The **Implied Volatility** is that value $\\sigma$ that must be inserted into the Black-Scholes (BS) formula in order to retrieve the option price quoted in the market:
$$ BS(S, K, T, r, \sigma) = P  $$
where $S$ is the underlying spot price, $K$ is the strike, $T$ time to maturity, $r$ risk-free interest rate and $P$ the option price quoted in the market. All these quantities are **observable**.


In [1]:
from functions.MERTONpricer import Merton_pricer
from functions.BSpricer import BS_pricer
from functions.KOUpricer import Kou_pricer

import numpy as np
import pandas as pd
import scipy as scp
import scipy.stats as ss
import scipy.optimize as scpo


Let's retrieve the historic prices for european call and put options starting from 2016-01-20 and expiring 1 year later.

In [2]:
df_call = pd.read_csv('data/options_spx_call_2016.csv')
df_put = pd.read_csv('data/options_spx_put_2016.csv')

df_call['C_Midpoint'] = abs(df_call['C_BID'] + df_call['C_ASK']) / 2
df_put['P_Midpoint'] = abs(df_put['P_BID'] + df_put['P_ASK']) / 2
df_call['C_Spread'] = df_call['C_BID'] - df_call['C_ASK']
df_put['P_Spread'] = df_put['P_BID'] - df_put['P_ASK']

r = 0.1
sigma = 0.5
S0 = df_call.iloc[0]['UNDERLYING_LAST']
T = 1
call_strikes = df_call['STRIKE']    # array of K
put_strikes = df_put['STRIKE']
exercise = 'european'

call_prices = df_call['C_Midpoint']
put_prices = df_put['P_Midpoint']

Let's initialize an object of class BS_pricer which is able to find the theoretical price of the options, given the parameters.

In [3]:
BS = BS_pricer(S0=S0, r=r, sigma=sigma, ttm=T, exercise=exercise, K=None)
Merton = Merton_pricer(S0=S0, K=None, ttm=T, r=r, sigma=sigma, lambd=0.8, meanJ=0.6, stdJ=0.2, exercise=exercise)
Kou = Kou_pricer(S0=S0, K=None, ttm=T, r=r, sigma=sigma, lambd=1, p=0.4, eta1=10, eta2=5, exercise=exercise)

call_th_prices = pd.DataFrame(columns=['BlackScholes', 'Merton', 'Kou'])

for i, K in enumerate(call_strikes):
    bs = BS.closed_formula_call(K)
    mert = Merton.closed_formula_call(K)
    kou = Kou.closed_formula_call(K)
    call_th_prices.loc[i] = [bs, mert, kou]

print(f'Theoretical call options prices:')
print(call_th_prices)


Theoretical call options prices:
    BlackScholes      Merton         Kou
0     519.267776  621.720722  545.888779
1     506.924480  610.291830  533.846489
2     494.841442  599.052799  522.044610
3     483.015579  588.001572  510.480268
4     471.443681  577.136062  499.150498
5     460.122426  566.454151  488.052252
6     449.048389  555.953702  477.182409
7     438.218057  545.632552  466.537785
8     427.627837  535.488521  456.115137
9     417.274071  525.519413  445.911177
10    407.153041  515.723016  435.922576
11    397.260982  506.097111  426.145970
12    387.594089  496.639464  416.577970
13    378.148525  487.347840  407.215165


PUT PRICES:

In [4]:
put_th_prices = pd.DataFrame(columns=['BlackScholes', 'Merton', 'Kou'])

for i, K in enumerate(put_strikes):
    bs = BS.closed_formula_put(K)
    mert = Merton.closed_formula_put(K)
    kou = Kou.closed_formula_put(K)
    put_th_prices.loc[i] = [bs, mert, kou]

print(f'Theoretical call options prices:')
print(call_th_prices)

Theoretical call options prices:
    BlackScholes      Merton         Kou
0     519.267776  621.720722  545.888779
1     506.924480  610.291830  533.846489
2     494.841442  599.052799  522.044610
3     483.015579  588.001572  510.480268
4     471.443681  577.136062  499.150498
5     460.122426  566.454151  488.052252
6     449.048389  555.953702  477.182409
7     438.218057  545.632552  466.537785
8     427.627837  535.488521  456.115137
9     417.274071  525.519413  445.911177
10    407.153041  515.723016  435.922576
11    397.260982  506.097111  426.145970
12    387.594089  496.639464  416.577970
13    378.148525  487.347840  407.215165


In [5]:
def implied_volatility(price, S0, K, T, r, type_o, method, disp=True ):
    """ Returns Implied volatility
        methods:  fsolve (default) or brent
    """

    def obj_fun(vol):
        return price - BS.BlackScholes(type_o=type_o, S0=S0, K=K, ttm=T, r=r, sigma=vol)

    if method == 'brent':
        x, r = scpo.brentq( obj_fun, a=1e-15, b=500, full_output=True)
        if r.converged:
            return x

    if method =='fsolve':
        X0 = [0.03, 0.08, 0.1, 0.15, 0.2, 0.3, 0.5, 1, 2]   # set of initial guess points for imp.vol.
        for x0 in X0:
            x, _, solved, _ = scpo.fsolve(obj_fun, x0, full_output=True, xtol=1e-8)
            if solved == 1:
                return x[0]

    if method == 'newton':
        x0 = 0.3

        def obj_fun_squared(vol):
            return obj_fun(vol)**2

        # print('[σ] = ', x0, "Objective Function value: ", obj_fun_squared(x0))
        result = scpo.fmin(obj_fun_squared, x0, ftol=1e-10, full_output=0, disp=0)
        return result[0]


    if disp:
        return -1

Let's now compute the implied volatilities from the true market prices *call_prices*.

In [6]:
IV_BS = []; IV_M = []; IV_K = []

for i in range(len(call_prices)):
    IV_BS.append(implied_volatility(call_th_prices['BlackScholes'].values[i], S0=S0, K=call_strikes[i], T=T, r=r, type_o='call', method='newton') )
    IV_M.append(implied_volatility(call_th_prices['Merton'].values[i], S0=S0, K = call_strikes[i], T=T, r=r, type_o='call', method='fsolve'))
    IV_K.append(implied_volatility(call_th_prices['Kou'].values[i], S0=S0, K = call_strikes[i], T=T, r=r, type_o='call', method='fsolve'))

imp_vol_call = pd.DataFrame(columns=['BlackScholes', 'Merton', 'Kou'])
imp_vol_call['BlackScholes'] = IV_BS
imp_vol_call['Merton'] = IV_M
imp_vol_call['Kou'] = IV_K

print(f'Implied volatilities found by each model for call options:\n{imp_vol_call}')

Implied volatilities found by each model for call options:
    BlackScholes    Merton       Kou
0            0.5  0.667278  0.543615
1            0.5  0.666211  0.543361
2            0.5  0.665162  0.543115
3            0.5  0.664130  0.542877
4            0.5  0.663116  0.542646
5            0.5  0.662117  0.542421
6            0.5  0.661136  0.542204
7            0.5  0.660170  0.541993
8            0.5  0.659219  0.541788
9            0.5  0.658284  0.541589
10           0.5  0.657363  0.541396
11           0.5  0.656457  0.541208
12           0.5  0.655565  0.541026
13           0.5  0.654687  0.540848


In [9]:
IV_BS = []; IV_M = []; IV_K = []

for i in range(len(put_prices)):
    IV_BS.append(implied_volatility(put_th_prices['BlackScholes'].values[i], S0=S0, K=put_strikes[i], T=T, r=r, type_o='put', method='newton') )
    IV_M.append(implied_volatility(put_th_prices['Merton'].values[i], S0=S0, K = put_strikes[i], T=T, r=r, type_o='put', method='fsolve'))
    IV_K.append(implied_volatility(put_th_prices['Kou'].values[i], S0=S0, K = put_strikes[i], T=T, r=r, type_o='put', method='fsolve'))

imp_vol_put = pd.DataFrame(columns=['BlackScholes', 'Merton', 'Kou'])
imp_vol_put['BlackScholes'] = IV_BS
imp_vol_put['Merton'] = IV_M
imp_vol_put['Kou'] = IV_K

print(f'Implied volatilities found by each model for put options:\n {imp_vol_put}')

Implied volatilities found by each model for put options:
     BlackScholes    Merton       Kou
0            0.5  0.667278  0.543615
1            0.5  0.666211  0.543361
2            0.5  0.665162  0.543115
3            0.5  0.664130  0.542877
4            0.5  0.663116  0.542646
5            0.5  0.662117  0.542421
6            0.5  0.661136  0.542204
7            0.5  0.660170  0.541993
8            0.5  0.659219  0.541788
9            0.5  0.658284  0.541589
10           0.5  0.657363  0.541396
11           0.5  0.656457  0.541208
12           0.5  0.655565  0.541026
13           0.5  0.654687  0.540848


## Calibration of Merton Jump Diffusion parameters
First we'll initialize a starting point in the array $x0 = [σ, λ, m, v]$
and we set bounds for the 4 parameters. Then we use the method
1. **curve_fit** of scipy.optimize. It uses the trf method when the parameters are bounded and the Levemberg-Marquadt method for unbounded parameters.

In [10]:
x0 = [0.15, 1, 0.1, 1] # initial guess: [σ, λ, m, v]
bounds = ( [0, 0, -10, 0], [np.inf, np.inf, 10, 5] )

def f_Mert(x, sigma, lambd, meanJ, stdJ):
    Mert = Merton_pricer(S0=S0, K=x, ttm=T, r=r, sigma=sigma, lambd=lambd, meanJ=meanJ, stdJ=stdJ, exercise=exercise)
    return Mert.closed_formula_call(x)
#

- Method 1. curve_fit of scipy.optimize. It uses the trf method when the parameters are bounded and the Levemberg-Marquadt method for unbounded parameters.

In [13]:
res1 = scpo.curve_fit(f_Mert, call_strikes, call_prices, p0=x0, bounds=bounds)

sigt = round(res1[0][0],5)
lambdt = round(res1[0][1],2)
mt = round(res1[0][2],2)
vt = round(res1[0][3],5)

print('METHOD 1: CURVE_FIT (trf)')
print('> Calibrated Volatlity [σ] = ', sigt)
print('> Calibrated Jump intensity [λ] = ', lambdt)
print('> Calibrated Jump Mean = ', mt)
print('> Calibrated Jump St. dev.  = ', vt)

METHOD 1: CURVE_FIT (trf)
> Calibrated Volatlity [σ] =  2e-05
> Calibrated Jump intensity [λ] =  0.58
> Calibrated Jump Mean =  1.08
> Calibrated Jump St. dev.  =  0.02333


In [15]:
res1_2 = scpo.curve_fit(f_Mert, call_strikes, call_prices, p0=x0)

sigt2 = round(res1_2[0][0],5)
lambdt2 = round(res1_2[0][1],2)
mt2 = round(res1_2[0][2],2)
vt2 = round(res1_2[0][3],5)

print('METHOD 1.2: CURVE_FIT (Levemberg-Marquadt, no bounds)')
print('> Calibrated Volatlity [σ] = ', sigt2)
print('> Calibrated Jump intensity [λ] = ', lambdt2)
print('> Calibrated Jump Mean = ', mt2)
print('> Calibrated Jump St. dev.  = ', vt2)

C:\Users\ornel\OneDrive\Desktop\TESI\PythonProjects\Margin-Loans-Pricing\python-modules\functions\MERTONpricer.py:40: RuntimeWarning: invalid value encountered in log
  return self.r - self.lambd * (self.meanJ - 1) + (k * np.log(self.meanJ)) / self.ttm


METHOD 1.2: CURVE_FIT (Levemberg-Marquadt, no bounds)
> Calibrated Volatlity [σ] =  -0.14044
> Calibrated Jump intensity [λ] =  -0.07
> Calibrated Jump Mean =  0.0
> Calibrated Jump St. dev.  =  0.2125


c:\users\ornel\appdata\local\programs\python\python38\lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


- Method 2. minimize(method=’SLSQP’)

In [16]:
x0 = [0.2, 0.8, 0.1, 1] # initial guess: [σ, λ, m, v]
bounds = [(0.01, 3), (0, np.inf), (1e-5, 5),  (0, 5)]

# Define the objective function
def obj_function(x, K, call_prices):
    sigm, lamb, mean, std = x
    Mert = Merton_pricer(S0=S0, K=K, ttm=T, r=r, sigma=sigm, lambd=lamb, meanJ=mean, stdJ=std, exercise=exercise)
    return np.sum((Mert.closed_formula_call(K) - call_prices) ** 2)

additional_args = (call_strikes, call_prices)

res2 = scpo.minimize(obj_function, x0, args=additional_args,  method='SLSQP', bounds=bounds, tol=1e-20)

print(res2)
sigt, lambdt, mt, vt = res2.x

# Print the results
print('> METHOD 2: MINIMIZE (SLSQP)')
print('Calibrated Volatility [σ] =', round(sigt, 5))
print('Calibrated Jump Intensity [λ] =', round(lambdt, 2))
print('Calibrated Jump Mean [m] =', round(mt, 2))
print('Calibrated Jump St. dev. [v] =', round(vt, 2))

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1831705.969667116
       x: [ 1.452e-01  5.693e-01  1.513e+00  7.117e-01]
     nit: 6
     jac: [ 1.981e+06  4.962e+06  3.445e+06  3.245e+06]
    nfev: 12
    njev: 2
> METHOD 2: MINIMIZE (SLSQP)
Calibrated Volatility [σ] = 0.14522
Calibrated Jump Intensity [λ] = 0.57
Calibrated Jump Mean [m] = 1.51
Calibrated Jump St. dev. [v] = 0.71


## Calibration of Kou Jump Diffusion parameters

In [19]:
#### KOU example from Kou 2002 (footnote 9) to check correctness
Kou = Kou_pricer(100, 98, 0.5, 0.05, 0.16, 1, 0.4, 10, 5, exercise)
print(Kou.closed_formula_call(98))
print(Kou.closed_formula_put(101))

9.14731730389542
5.910755746956134
